In [1]:
from vc_clothes_dataset import VCClothesDataset, build_transforms
from face_feature_extractor import FaceReIDModel
from face_feature_extractor_wrapper import FaceReIDTrainWrapper
from trainer import Trainer

from torch.utils.data import Dataset, DataLoader
import torch
import torch.optim as optim

In [2]:
DATA_ROOT = "./data/vc_clothes"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
LR = 0.00035
EPOCHS = 10

In [3]:
# Tworzenie transformacji
train_transforms = build_transforms(is_train=True)
val_transforms = build_transforms(is_train=False)

# Instancja datasetu treningowego
# VC-Clothes ma 9,449 obrazów treningowych [cite: 194]
train_set = VCClothesDataset(DATA_ROOT, mode="train", transform=train_transforms)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)

# Instancja datasetu testowego (Query)
# VC-Clothes ma 1,020 obrazów query [cite: 194]
query_set = VCClothesDataset(DATA_ROOT, mode="query", transform=val_transforms)
test_loader = DataLoader(query_set, batch_size=32, shuffle=False, num_workers=4)

print("\nPrzykładowy element batcha:")
imgs, pids, cams, clothes = next(iter(train_loader))
print(f"Shape obrazów: {imgs.shape}")  # Oczekiwane [32, 3, 256, 128]
print(f"PIDs: {pids}")
print(f"Clothes IDs: {clothes}")

Załadowano zbiór VC-Clothes (train):
  Liczba obrazów: 9449
  Liczba unikalnych ID: 256
  Liczba kamer: 4
  Liczba unikalnych ubrań: 3
Załadowano zbiór VC-Clothes (query):
  Liczba obrazów: 1020
  Liczba unikalnych ID: 256
  Liczba kamer: 4
  Liczba unikalnych ubrań: 3

Przykładowy element batcha:
Shape obrazów: torch.Size([32, 3, 256, 128])
PIDs: tensor([202, 155,  57, 225, 143,  16, 150, 147, 201,  14,  49, 208, 130, 129,
        163, 190,  50, 243, 192,  10, 127, 225, 176, 177, 200,  21, 214,  83,
         49, 254, 184,  34])
Clothes IDs: tensor([2, 1, 3, 1, 2, 1, 1, 1, 1, 1, 3, 2, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1,
        1, 3, 1, 1, 2, 1, 1, 1])


In [5]:
base_model = FaceReIDModel(device=DEVICE)
train_model = FaceReIDTrainWrapper(base_model, num_classes=256)

optimizer = optim.Adam(train_model.parameters(), lr=LR, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
trainer = Trainer(
    model=train_model,
    train_loader=train_loader,
    val_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE,
)

In [7]:
results = trainer.fit(num_epochs=EPOCHS)
results

Rozpoczęcie treningu na urządzeniu: cpu


Train Epoka 1:   0%|          | 0/296 [00:12<?, ?it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors